# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

* ① 印出第一頁列表所有文章的「作者」「標題」「時間」
* ② 印出最新文章的「作者」「標題」「時間」(從列表內頁+時間排序)
* ③ 試著爬爬看八卦版的文章 (繞過cookie)

### ① 印出第一頁列表所有文章的「作者」「標題」「時間」# random order

In [84]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)
soup = BeautifulSoup(r.text, "html5lib")
for d in soup.find_all(class_="r-ent"): #list
    #list內只能用find，因為他是一個一個找
    print(d.find(class_="title").text.replace('\t','').replace('\n', '')) 
    print("作者:",d.find(class_="author").text)
    print("日期",d.find(class_="date").text)

[花邊] 勇士全隊誰最像喬丹？柯爾答案不是柯瑞
作者: Vicky
日期  4/03
[花邊] 挑湖人舊比賽當教材 黃蜂上下讚嘆拜能昔日身手
作者: thnlkj0665
日期  4/03
Re: [情報] Butler 透露被公牛交易後在巴黎花了100萬
作者: lwy012018
日期  4/03
[專欄] 半人半神將回歸凡間 卡特值得更好的謝幕
作者: hvariables
日期  4/03
Re: [新聞] 林書豪出關哈登照 豪迷驚：怎變大叔
作者: NoAfraid
日期  4/03
[新聞] 隔離生活好忙碌 海沃德陪女兒看《冰雪奇緣
作者: Gotham
日期  4/03
[新聞] 看好狀元威廉森  MVP與名人堂是他的上限
作者: PttGod
日期  4/03
[花邊] 詹皇和喬丹的最大差別 前隊友休斯：垃圾話
作者: Intelnet
日期  4/03
[情報] Kobe Duncan Garnett入選名人堂
作者: Bainite
日期  4/04
[BOX ] #1 Durant 62:78 #16 Jones Jr.
作者: Malpais
日期  4/04
[花邊] ESPN秀陣容圖：五個Curry打五個字母哥
作者: PttGod
日期  4/04
[公告] 板規v6.8
作者: Vedan
日期  5/24
[公告] 版主上任相關事項 
作者: qazwsx879345
日期 10/25
[情報] SEASON Schedule 賽程 March 19–20
作者: Acetoxy
日期  3/02
[公告] 第一次被退文，可在三天後刪除退文
作者: Vedan
日期  3/09
[公告] 板務說明 (發錢)
作者: Vedan
日期  4/03


### ② 印出最新文章的「作者」「標題」「時間」

In [85]:
#因為原始的資料難以判斷「新/舊」，因此我們必須進入內頁抓取詳細的時間，最終存成 List of Dict 的形式再自行排序。

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['Title'] = d.text.replace('\t', '').replace('\n', '')
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['Author'] = r.find(class_='article-meta-value').text
        post['Time'] = r.find_all(class_='article-meta-value')[-1].text
        posts.append(post)
    except:
        continue

In [86]:
posts = sorted(posts, key= lambda x: x['Time'])
posts #發現時間並未正確排序，因為time沒有轉datetime，所以是照字母排序

[{'Title': '[公告] 板務說明 (發錢)',
  'Author': 'Vedan (味丹)',
  'Time': 'Fri Apr  3 01:28:00 2020'},
 {'Title': '[花邊] 勇士全隊誰最像喬丹？柯爾答案不是柯瑞',
  'Author': 'Vicky (維琪)',
  'Time': 'Fri Apr  3 19:20:31 2020'},
 {'Title': '[花邊] 挑湖人舊比賽當教材 黃蜂上下讚嘆拜能昔日身手',
  'Author': 'thnlkj0665 (灰色地帶)',
  'Time': 'Fri Apr  3 21:25:21 2020'},
 {'Title': 'Re: [情報] Butler 透露被公牛交易後在巴黎花了100萬',
  'Author': 'lwy012018 (shift man)',
  'Time': 'Fri Apr  3 22:48:16 2020'},
 {'Title': '[專欄] 半人半神將回歸凡間 卡特值得更好的謝幕',
  'Author': 'hvariables (Speculative Male)',
  'Time': 'Fri Apr  3 23:07:59 2020'},
 {'Title': 'Re: [新聞] 林書豪出關哈登照 豪迷驚：怎變大叔',
  'Author': 'NoAfraid (小子難纏 不得安寧)',
  'Time': 'Fri Apr  3 23:08:57 2020'},
 {'Title': '[新聞] 隔離生活好忙碌 海沃德陪女兒看《冰雪奇緣',
  'Author': 'Gotham (萬惡之城)',
  'Time': 'Fri Apr  3 23:57:34 2020'},
 {'Title': '[新聞] 看好狀元威廉森  MVP與名人堂是他的上限',
  'Author': 'PttGod (PttGod)',
  'Time': 'Fri Apr  3 23:59:14 2020'},
 {'Title': '[花邊] 詹皇和喬丹的最大差別 前隊友休斯：垃圾話',
  'Author': 'Intelnet ()',
  'Time': 'Fri Apr  3 23:59:45 2020'},
 

In [97]:
#重寫一次
#時間格式：Fri Apr  3 01:28:00 2020 --> %a %b %d %H:%M:%S %Y
from datetime import datetime

In [98]:
url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)
soup = BeautifulSoup(r.text, "html5lib")
posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['Title'] = d.text.replace('\t', '').replace('\n', '')
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['Author'] = r.find(class_='article-meta-value').text
        post['Time'] = datetime.strptime(r.find_all(class_='article-meta-value')[-1].text,"%a %b  %d %H:%M:%S %Y")
        posts.append(post)
    except:
        continue

In [101]:
posts=sorted(posts, key= lambda x: x['Time'],reverse=True)
posts

[{'Title': '[情報] 公牛尋找新GM，全權負責籃球決策',
  'Author': 'driftcreator (北美漂流中)',
  'Time': datetime.datetime(2020, 4, 4, 11, 13, 16)},
 {'Title': '[花邊] ESPN秀陣容圖：五個Curry打五個字母哥',
  'Author': 'PttGod (PttGod)',
  'Time': datetime.datetime(2020, 4, 4, 10, 37, 35)},
 {'Title': '[BOX ] #1 Durant 62:78 #16 Jones Jr.',
  'Author': 'Malpais (Malpais)',
  'Time': datetime.datetime(2020, 4, 4, 9, 18, 8)},
 {'Title': '[情報] Kobe Duncan Garnett入選名人堂',
  'Author': 'Bainite (黑羽毛 ┐(′д`)┌)',
  'Time': datetime.datetime(2020, 4, 4, 4, 48, 26)},
 {'Title': '[花邊] 詹皇和喬丹的最大差別 前隊友休斯：垃圾話',
  'Author': 'Intelnet ()',
  'Time': datetime.datetime(2020, 4, 3, 23, 59, 45)},
 {'Title': '[新聞] 看好狀元威廉森  MVP與名人堂是他的上限',
  'Author': 'PttGod (PttGod)',
  'Time': datetime.datetime(2020, 4, 3, 23, 59, 14)},
 {'Title': '[新聞] 隔離生活好忙碌 海沃德陪女兒看《冰雪奇緣',
  'Author': 'Gotham (萬惡之城)',
  'Time': datetime.datetime(2020, 4, 3, 23, 57, 34)},
 {'Title': 'Re: [新聞] 林書豪出關哈登照 豪迷驚：怎變大叔',
  'Author': 'NoAfraid (小子難纏 不得安寧)',
  'Time': datetime.datetime(20

In [103]:
posts[0]

{'Title': '[情報] 公牛尋找新GM，全權負責籃球決策',
 'Author': 'driftcreator (北美漂流中)',
 'Time': datetime.datetime(2020, 4, 4, 11, 13, 16)}

### ③ 試著爬爬看看八卦版的文章 (cookie)

In [111]:
# 八卦版會跳轉到一個「十八歲的同意驗證頁面」須利用cookie繞過驗證
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['url']='https://www.ptt.cc'+d.find('a')['href']
        post['Title'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['Author'] = r.find(class_='article-meta-value').text
        post['Time'] = datetime.strptime(r.find_all(class_='article-meta-value')[-1].text, "%a %b %d %H:%M:%S %Y")
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['Time'],reverse=True)

In [112]:
#最新文章
posts[0]

{'url': 'https://www.ptt.cc/bbs/Gossiping/M.1585970705.A.D17.html',
 'Title': '[問卦] 兔子牙的八卦?',
 'Author': 'Godfrey0216 (Godfrey)',
 'Time': datetime.datetime(2020, 4, 4, 11, 25, 3)}

In [139]:
#印出最新文章內容
url =posts[0]['url']
r = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(r.text, "html5lib")

In [144]:
soup.find(id="main-content").text

'作者Godfrey0216 (Godfrey)看板Gossiping標題[問卦] 兔子牙的八卦?時間Sat Apr  4 11:25:03 2020\n\n\n      馮提莫退隱之後,下一個崛起的就是兔子牙了!!可是感覺兔子牙是走\n\n      親合甜美路線,我是覺得馮好像小贏一點,覺得這後浪有點沒推翻前浪~~\n\n--\n※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 61.57.109.86 (臺灣)\n※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1585970705.A.D17.html\n噓 Silent: 這裡是八 04/04 11:25\n→ love0504: 誰 04/04 11:25\n推 eric88888: 五樓肛門長牙 04/04 11:25\n噓 omfg5487: 沒卦 04/04 11:25\n→ ohrring: 暴牙啊幹 04/04 11:27\n'